___
# Academics of Py
#### Submitted by Kim Harrison
##### UCIRV201902DATA3

___
## PyCitySchools Findings
*In general, Charter Schools offered the best value in terms of Cost of Eductation.  Charter schools spent an average of \\$599.50 per student with an overall passing rate of 95.10%.  District schools spent more on average (\\$643.57 per student) with an average passing rate of 73.67%. <br><br>
*The schools with a spending range of \\$600-\\$625 per student yielded the highets overall passing rate of 95.27%.  There was a nearly 20% decrease in overall passing rate (77.28) when spending \\$625-$650 per student.  This follows with the trend that Large schools experience a similar drop off in overall passing rate (95.20% for medium compared to 76.36% for large), suggesting that the drop off in performance is may be related to the size of the school versus the spending rates.<br><br>
*Overall, math seems to be the more challenging subject for all students, in all grades, in all school types.<br>
___

In [44]:
#Import Dependencies
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Load csv files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

#Merge csv files into dataframe
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary
___

In [45]:
#calculate district summary statistics
total_schools = len(school_data_complete['school_name'].unique())
total_students = len(school_data_complete['Student ID'].unique())
total_budget = school_data['budget'].sum()
math_score_avg = school_data_complete['math_score'].mean()
reading_score_avg = school_data_complete['reading_score'].mean()
overall_passing_pct = (math_score_avg + reading_score_avg) / 2
math_passing_pct = (len(school_data_complete.loc[school_data_complete['math_score'] >= 70])) / total_students * 100
reading_passing_pct = (len(school_data_complete.loc[school_data_complete['reading_score'] >= 70])) / total_students * 100

#store district summary statistics into summary dataframe
district_summary = pd.DataFrame({
    'Total Schools': [total_schools], 
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Average Math Score': [math_score_avg],
    'Average Reading Score' : [reading_score_avg],
    '% Passing Math': [math_passing_pct],
    '% Passing Reading': [reading_passing_pct],
    'Overall Passing %': [overall_passing_pct]
})

#set up format mapping for dataframe
format_mapping = {
    'Total Budget': '${:,.0f}',
    'Average Math Score': '{:,.2f}',
    'Average Reading Score': '{:,.2f}',
    '% Passing Math': '{:,.2f}',
    '% Passing Reading': '{:,.2f}',
    'Overall Passing %': '{:,.2f}',
    'Total Students' : '{:,}',
}    

#assign mapping values to data frame
for key, value in format_mapping.items():
    district_summary[key] = district_summary[key].apply(value.format)

#print district summary dataframe
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,80.43


## School Summary
---

In [46]:
#create new data frame of school info to modify with calculations
school_data_df = school_data

#calculate the money spent per student
school_data_df['Per Student Budget'] = school_data_df['budget'] / school_data_df['size']

#calculate averages for reading score and math score for grouped dataset
school_summary_avgs = school_data_complete.groupby('school_name') \
                                .agg({'reading_score':'mean', 'math_score':'mean'}) \
                                .rename(columns={'reading_score':'Average Reading Score','math_score': 'Average Math Score'})

#filter data set for students at each school with passing students in both math and reading
school_scores_passing_math = school_data_complete[['school_name','math_score']].loc[school_data_complete['math_score'] >= 70]
school_scores_passing_reading = school_data_complete[['school_name','reading_score']].loc[school_data_complete['reading_score'] >= 70]

#group & count passing math grades; rename column as % passing (to be calcualte in next steps)
school_passing_math_count = school_scores_passing_math.groupby('school_name') \
                        .agg({'math_score':'count'}) \
                        .rename(columns={'math_score': '% Passing Math'})
school_passing_reading_count = school_scores_passing_reading.groupby('school_name') \
                        .agg({'reading_score':'count'}) \
                        .rename(columns={'reading_score': '% Passing Reading'})

#calculate total number of students at each school
school_size = school_data_complete[['school_name', 'size']].groupby('school_name').count()

#calculate passing percentages of each subject
school_passing_pct_math = school_passing_math_count.div(school_size['size'], axis ='index') * 100
school_passing_pct_reading = school_passing_reading_count.div(school_size['size'], axis ='index') * 100

#combine into summary data set and add column for average passing rate
school_passing_df = pd.concat([school_passing_pct_reading, school_passing_pct_math], axis=1)
school_passing_df['Overall Passing Rate'] = (school_passing_df['% Passing Math'] + school_passing_df['% Passing Reading'])/2

#merge dataframes & print results
school_summary1_df = pd.merge(school_data_df, school_summary_avgs, how="left", on=["school_name", "school_name"])
school_summary2_df = pd.merge(school_summary1_df, school_passing_df, how="left", on=["school_name", "school_name"])

school_summary2_df = school_summary2_df.rename(columns={'school_name':'School Name', 'type': 'Type', \
                                                      'size': 'Total Students','budget':'Total Budget'}) \
                                     .set_index('School Name')

del school_summary2_df['School ID']

#format dataframe
format_mapping = {
    'Total Budget': '${:,.0f}',
     'Average Math Score': '{:,.2f}',
     'Average Reading Score': '{:,.2f}',
     '% Passing Math': '{:,.2f}',
     '% Passing Reading': '{:,.2f}',
     'Overall Passing Rate': '{:,.2f}',
     'Total Students' : '{:}',
     'Per Student Budget' : '${:,.0f}' ,
}    

for key, value in format_mapping.items():
    school_summary2_df[key] = school_summary2_df[key].apply(value.format)

#print school summary data frame
school_summary2_df

,Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,"$1,910,635",$655,81.18,76.63,81.32,65.68,73.50
Figueroa High School,District,2949,"$1,884,411",$639,81.16,76.71,80.74,65.99,73.36
Shelton High School,Charter,1761,"$1,056,600",$600,83.73,83.36,95.85,93.87,94.86
Hernandez High School,District,4635,"$3,022,020",$652,80.93,77.29,80.86,66.75,73.81
Griffin High School,Charter,1468,"$917,500",$625,83.82,83.35,97.14,93.39,95.27
Wilson High School,Charter,2283,"$1,319,574",$578,83.99,83.27,96.54,93.87,95.20
Cabrera High School,Charter,1858,"$1,081,356",$582,83.98,83.06,97.04,94.13,95.59
Bailey High School,District,4976,"$3,124,928",$628,81.03,77.05,81.93,66.68,74.31
Holden High School,Charter,427,"$248,087",$581,83.81,83.80,96.25,92.51,94.38


## Top Performing Schools (By Passing Rate)
___

In [47]:
#Sort values in descending order to determine top 5 schools by passing rate
top_performer = school_summary2_df.sort_values('Overall Passing Rate', ascending=False)
top_performer.head()

,Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,83.98,83.06,97.04,94.13,95.59
Thomas High School,Charter,1635,"$1,043,130",$638,83.85,83.42,97.31,93.27,95.29
Griffin High School,Charter,1468,"$917,500",$625,83.82,83.35,97.14,93.39,95.27
Pena High School,Charter,962,"$585,858",$609,84.04,83.84,95.95,94.59,95.27
Wilson High School,Charter,2283,"$1,319,574",$578,83.99,83.27,96.54,93.87,95.20


## Bottom Performing Schools (By Passing Rate)
___

In [48]:
#Sort values in ascending order to determine top 5 schools by passing rate
bottom_performer = school_summary2_df.sort_values('Overall Passing Rate', ascending=True)
bottom_performer.head()

,Type,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637,80.74,76.84,80.22,66.37,73.29
Figueroa High School,District,2949,"$1,884,411",$639,81.16,76.71,80.74,65.99,73.36
Huang High School,District,2917,"$1,910,635",$655,81.18,76.63,81.32,65.68,73.50
Johnson High School,District,4761,"$3,094,650",$650,80.97,77.07,81.22,66.06,73.64
Ford High School,District,2739,"$1,763,916",$644,80.75,77.10,79.30,68.31,73.80


## Math Scores by Grade
___

In [49]:
#create data frame for math data and group by grade
student_data_math = student_data[['school_name','grade','math_score']].groupby(['grade', 'school_name'])

#calculate average for each grade at each school
math_grade_avg = student_data_math.mean()

#store averages into series to be concatenated
school_names = student_data['school_name']
math_fresh = math_grade_avg.loc['9th']
math_soph = math_grade_avg.loc['10th']
math_junior = math_grade_avg.loc['11th']
math_senior = math_grade_avg.loc['12th']

#concatenate series into dataframe; label columns
math_avg_total = pd.concat([math_fresh, math_soph, math_junior, math_senior], axis=1)
grades = ['9th', '10th', '11th', '12th']
math_avg_total.columns = grades
del math_avg_total.index.name

#print math scores by grade
pd.options.display.float_format = '{:,.2f}'.format
math_avg_total

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Scores by Grade
___

In [50]:
#create data frame for math data and group by grade
student_data_reading = student_data[['school_name','grade','reading_score']].groupby(['grade', 'school_name'])
reading_grade_avg = student_data_reading.mean()

#store averages into series to be concatenated
reading_fresh = reading_grade_avg.loc['9th']
reading_soph = reading_grade_avg.loc['10th']
reading_junior = reading_grade_avg.loc['11th']
reading_senior = reading_grade_avg.loc['12th']

#concatenate series into dataframe; label columns
reading_avg_total = pd.concat([reading_fresh, reading_soph, reading_junior, reading_senior], axis=1)
grades = ['9th', '10th', '11th', '12th']
reading_avg_total.columns = grades
del reading_avg_total.index.name

#print reading scores by grade
reading_avg_total

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending
___

In [51]:
#Establish bins & labels for spending
bins = [0, 600, 625, 650, 700]
labels = ["< \$600", "\$600-$625", "\$625-$650", "\$675 +"]

#create & bin new dataframe (need to convert data back to numeric since it had been formatted previously)
school_spend_bin_formatted = school_summary2_df[['Per Student Budget', 'Average Reading Score','Average Math Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
school_spend_bin_formatted['Per Student Budget'] = school_spend_bin_formatted['Per Student Budget'].str.replace('$','')
school_spend_bin = school_spend_bin_formatted.apply(pd.to_numeric)
school_spend_bin['Spending Ranges'] = pd.cut(school_spend_bin['Per Student Budget'], bins, labels=labels)

#group dataframe by spending ranges & calcualte passing rates
school_spend_bin = school_spend_bin.groupby('Spending Ranges')
school_spend_bin_df = school_spend_bin[['Average Math Score', "Average Reading Score", '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]

#print dataframe of school scores by spending
school_spend_bin_df.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges,,,,,
< \$600,83.43,83.89,93.54,96.46,95.00
\$600-$625,83.59,83.93,93.99,96.55,95.27
\$625-$650,78.03,81.42,71.11,83.45,77.28
\$675 +,76.96,81.06,66.22,81.09,73.66


## Scores by School Size
___

In [52]:
#Establish bins & labels for school size by student
bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#create & bin new dataframe
school_size_bin_formatted = school_summary2_df[['Total Students', 'Average Reading Score','Average Math Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
school_size_bin = school_size_bin_formatted.apply(pd.to_numeric)
school_size_bin['School Size'] = pd.cut(school_size_bin['Total Students'], bins, labels=labels)

#group dataframe by school size & calcualte passing rates
school_size_bin = school_size_bin.groupby('School Size')
school_size_bin_df = school_size_bin[['Average Math Score', "Average Reading Score", '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]

#print dataframe of school scores by size
school_size_bin_df.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.87,93.60,96.79,95.20
Large (2000-5000),77.74,81.34,69.96,82.77,76.36


## Scores by School Type

In [53]:
#group dataframe by school type & calcualte passing rates
school_type_bin_formatted = school_summary2_df[['Average Reading Score','Average Math Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]

#convert passing rates to numerics
school_type_bin = school_type_bin_formatted.apply(pd.to_numeric)

#add school type column and grouby
school_type_bin['Type'] = school_summary2_df['Type']
school_type_bin_df = school_type_bin.groupby('Type')

#print dataframe of school scores by school type
school_type_bin_df.mean()

,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,
Charter,83.90,83.47,93.62,96.59,95.10
District,80.97,76.96,66.55,80.80,73.67
